In [1]:
import tensorflow as tf
import pyodbc
import numpy as np

with open('DATASET.npy', 'rb') as f:
    targets = np.load(f)
    notes = np.load(f)


In [2]:
train_dataset = tf.data.Dataset.from_tensor_slices((notes[0:99], targets[0:99]))
valid_dataset = tf.data.Dataset.from_tensor_slices((notes[100:131], targets[100:131]))

In [3]:
class Modele(tf.keras.Model):

    def __init__(self):
        super(Modele, self).__init__()
        self.conv_layer_one = tf.keras.layers.Conv1D(11, kernel_size=1, strides=1, activation='relu', padding='same', name='conv_layer_one')
        self.conv_layer_two = tf.keras.layers.Conv1D(11, kernel_size=3, strides=3, activation='relu', padding='same', name='conv_layer_two')
        self.flatten_layer = tf.keras.layers.Flatten(name='flatten_layer')
        self.input_layer = tf.keras.layers.Dense(11, activation='relu', name='input_layer')
        self.output_layer = tf.keras.layers.Dense(2, activation='softmax', name='output_layer')

    def call(self, notes):
        conv_layer_one = self.conv_layer_one(notes)
        conv_layer_two = self.conv_layer_two(conv_layer_one)
        flatten_layer = self.flatten_layer(conv_layer_two)
        input_layer = self.input_layer(flatten_layer)
        output_layer = self.output_layer(input_layer)
        return output_layer


In [23]:
model = Modele()
# model.compile(
#     loss="sparse_categorical_crossentropy",
#     optimizer="sgd",
#     metrics=["accuracy"]
# )
# model.fit(notes[20:120], targets[20:120], epochs=5)
# model.save("model_test", save_format="tf")



In [24]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [25]:
# loss "L'erreur"
train_loss = tf.keras.metrics.Mean(name="train_loss")
valid_loss = tf.keras.metrics.Mean(name="valid_loss")
# Accuracy "La précision"
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [27]:
@tf.function
def train_step(note, target):
    with tf.GradientTape() as tape:
        # faire une prediction sur tous le batch (groupe ou patie)
        prediction = model(note, training=True)
        # calculer l'erreur sur cette prediction
        loss = loss_object(target, prediction)
    # calcule le gradient qui respecte l'erreur  
    gradients = tape.gradient(loss, model.trainable_variables)
    # optimiser les poids du model selon le gradient calculé
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(target, prediction)

In [28]:
@tf.function
def test_step(note, target):
    # faire une prediction sur tous le batch (groupe ou patie)
    prediction = model(note)
    # calculer l'erreur sur cette prediction
    loss = loss_object(target, prediction)
    valid_loss(loss)
    valid_accuracy(target, prediction)

In [29]:
epoch= 10
batch_size = 10
b =0
model.compile(loss=loss_object, optimizer=optimizer)
for epoch in range(epoch):
    
    for notes_batch, tagets_batch in train_dataset.batch(batch_size):
        train_step(notes_batch, tagets_batch)
        template = '\r batch: {}/{}, erreur:  {}, Précision: {}'
        print(template.format(b, len(targets), train_loss.result(), train_accuracy.result()*100))
        b+= batch_size
        
        
    for notes_batch, tagets_batch in valid_dataset.batch(batch_size):
        test_step(notes_batch, tagets_batch)
        template = '\r epoch: {}, Test erreur:  {}, Test Précision: {}'
        print(template.format(epoch+1, valid_loss.result(), valid_accuracy.result()*100))
        
    print("------------")
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()

model.save("modele", save_format="tf")
# tf.saved_model.save(model, "saved_model_path") 

 batch: 0/132, erreur:  1.3748784065246582, Précision: 60.000003814697266
 batch: 10/132, erreur:  1.2804428339004517, Précision: 55.0
 batch: 20/132, erreur:  1.1457446813583374, Précision: 56.66666793823242
 batch: 30/132, erreur:  1.08902108669281, Précision: 60.000003814697266
 batch: 40/132, erreur:  1.0714702606201172, Précision: 58.0
 batch: 50/132, erreur:  0.9613497257232666, Précision: 63.33333206176758
 batch: 60/132, erreur:  0.8494951128959656, Précision: 68.5714340209961
 batch: 70/132, erreur:  0.8062482476234436, Précision: 70.0
 batch: 80/132, erreur:  0.8139694929122925, Précision: 68.8888931274414
 batch: 90/132, erreur:  0.824445366859436, Précision: 66.66667175292969
 epoch: 1, Test erreur:  0.45134982466697693, Test Précision: 80.0
 epoch: 1, Test erreur:  0.6358479857444763, Test Précision: 65.0
 epoch: 1, Test erreur:  0.5868467092514038, Test Précision: 66.66667175292969
 epoch: 1, Test erreur:  0.9381057024002075, Test Précision: 64.51612854003906
------------

 batch: 950/132, erreur:  0.6588780283927917, Précision: 63.33333206176758
 batch: 960/132, erreur:  0.6164923906326294, Précision: 68.5714340209961
 batch: 970/132, erreur:  0.6043953895568848, Précision: 70.0
 batch: 980/132, erreur:  0.6195074319839478, Précision: 67.77777862548828
 batch: 990/132, erreur:  0.6273093223571777, Précision: 65.65657043457031
 epoch: 10, Test erreur:  0.5027255415916443, Test Précision: 80.0
 epoch: 10, Test erreur:  0.5880314111709595, Test Précision: 65.0
 epoch: 10, Test erreur:  0.573170006275177, Test Précision: 66.66667175292969
 epoch: 10, Test erreur:  0.7052252292633057, Test Précision: 64.51612854003906
------------
INFO:tensorflow:Assets written to: modele\assets


In [36]:
load_model = tf.keras.models.load_model("modele")
pred = load_model.predict(notes[100:101])
print(pred, targets[100:101])
print(notes[100:101])

[[0.6078679 0.3921321]] [1]
[[[11.   10.   10.    5.   15.5  10.25  7.    3.75  8.5  18.    7.  ]
  [14.   12.   12.   13.   15.5  15.5  12.   12.5  15.5  18.   14.  ]
  [11.   10.   11.    5.   15.5  10.25  7.   13.    8.5  18.    7.  ]]]
